In [1]:
import numpy as np

### Define the Paul trap geometry

In [129]:
D_tube = 39.6e-3 # inner diameter of the CF nipple in m
d_elec = 3.175e-3 # diameter of an electrode in m (1/8" diameter rod)

print('Electrode diameter = {:.1f} mm'.format(d_elec*1e3))

r0_lim = D_tube/2./3.54 # largest practical quadrupole radius given cylinder size in m

print('Largest r_0 = {:.1f} mm'.format(r0_lim*1e3))

r_0 = (d_elec/2.)/1.1464

print('r_0 for given electrode = {:.1f} mm'.format(r_0*1e3))

g_wall = D_tube/2. - d_elec - 2*r_0 # gap between the electrode surface and the tube wall in m
# r_0 = (D_tube - 3*d_elec)/2. # distance between the trap center and electrode surface in m
g_elec = d_elec*(1/np.sqrt(2) - 1) + np.sqrt(2)*r_0 # gap between electrodes in m

print('Gap between electrode and wall = {:.1f} mm'.format(g_wall*1e3))
print('Gap between adjacent electrodes = {:.1f} mm'.format(g_elec*1e3))

d_ns = 166e-9 # 166 nm, diameter of a nanosphere in meters
r_ns = d_ns/2. # radius of a nanosphere in meters
V_ns = (4/3.)*np.pi*r_ns**3
rho_silica = 2.65e3 # density of silica, kg/m^3
m_ns = rho_silica*V_ns # mass of a nanosphere in kg

rho_eth = 789 # density of ethanol in kg/m^3
d_drop = 50e-6 # diameter of the ethanol drop in m
r_drop = d_drop/2. # radius of the ethanol drop in m
m_drop = rho_eth*(4/3)*np.pi*r_drop**3 # mass of the ethanol drop in kg

Electrode diameter = 3.2 mm
Largest r_0 = 5.6 mm
r_0 for given electrode = 1.4 mm
Gap between electrode and wall = 13.9 mm
Gap between adjacent electrodes = 1.0 mm


### Determine the trapping potential parameters

In [126]:
e = 1.609e-19 # unit charge in Coulombs
Q = 39*e # number of unit charges on the sphere
V_ac = 1000. # AC voltage applied between pairs of electrodes
U_dc = 0. # DC voltage applied between pairs of electrodes

# determine Mathieu parameters to check stability for given trap parameters
a_u = lambda f, U_dc, r_0: 4*Q*U_dc/m_ns/(2*np.pi*f)**2/r_0**2
q_u = lambda f, V_ac, r_0 : 4*Q*V_ac/m_ns/(2*np.pi*f)**2/r_0**2

# choose sample Mathieu equation parameters for stability
a_stab = 0
q_stab = 0.5

def V(Q, f=1000, m=m_ns):
    return m*(2*np.pi*f)**2*r_0**2*q_stab/(2*Q*e)

def f(Q, V=300, m=m_ns):
    return np.sqrt(V*2*(Q*e)/q_stab/m/(2*np.pi)**2/r_0**2)

In [193]:
print('RF amplitude of 300 V')
print('---------------------')
print('Ethanol drop with 1e3 charges: {:.1f} Hz'.format(f(1e3, m=m_drop)))
print('Ethanol drop with 1e6 charges: {:.1f} Hz'.format(f(1e6, m=m_drop)))
print('Ethanol drop with 1e9 charges: {:.1f} Hz'.format(f(1e9, m=m_drop)))
print('Nanosphere with 1e1 charges: {:.1f} Hz'.format(f(1e1, m=m_ns)))
print('Nanosphere with 1e2 charges: {:.1f} Hz'.format(f(1e2, m=m_ns)))
print('Nanosphere with 1e3 charges: {:.1f} Hz'.format(f(1e3, m=m_ns)))
print()
print('RF amplitude of 10 V')
print('---------------------')
print('Ethanol drop with 1e3 charges: {:.1f} Hz'.format(f(1e3, 10, m=m_drop)))
print('Ethanol drop with 1e6 charges: {:.1f} Hz'.format(f(1e6, 10, m=m_drop)))
print('Ethanol drop with 1e9 charges: {:.1f} Hz'.format(f(1e9, 10, m=m_drop)))
print('Nanosphere with 1e1 charges: {:.1f} Hz'.format(f(1e1, 10, m=m_ns)))
print('Nanosphere with 1e2 charges: {:.1f} Hz'.format(f(1e2, 10, m=m_ns)))
print('Nanosphere with 1e3 charges: {:.1f} Hz'.format(f(1e3, 10, m=m_ns)))

RF amplitude of 300 V
---------------------
Ethanol drop with 1e3 charges: 7.0 Hz
Ethanol drop with 1e6 charges: 222.2 Hz
Ethanol drop with 1e9 charges: 7027.8 Hz
Nanosphere with 1e1 charges: 2004.6 Hz
Nanosphere with 1e2 charges: 6339.1 Hz
Nanosphere with 1e3 charges: 20046.0 Hz

RF amplitude of 10 V
---------------------
Ethanol drop with 1e3 charges: 1.3 Hz
Ethanol drop with 1e6 charges: 40.6 Hz
Ethanol drop with 1e9 charges: 1283.1 Hz
Nanosphere with 1e1 charges: 366.0 Hz
Nanosphere with 1e2 charges: 1157.4 Hz
Nanosphere with 1e3 charges: 3659.9 Hz


### Can we see Rayleigh-scattered 532 nm laser light from a nanosphere?

In [190]:
P_0 = 40e-3 # incident power in W
lamb = 532e-9 # wavelength of the light in m
n_silica = 1.46 # index of refraction of the silica, from https://refractiveindex.info/?shelf=main&book=SiO2&page=Malitson
waist = 1e-3 # beam waist in m
A_det = (2.4e-3)**2 # detector sensitive area in m^2
I_dark = 5e-12 # detector dark current in A

def P_scatt(P_0, theta, R, r, n, A_det, waist):
    return (P_0/np.pi/waist**2)*A_det*(2*np.pi/lamb)**4*((n**2 - 1)/(n**2 + 2))**2\
        *r**6*(1 + np.cos(theta)**2)/2./R**2

P_det = P_scatt(P_0, np.pi/2, D_tube/2., r_ns, n_silica, (2.4e-3)**2, waist)
photosens = 0.36 # detector photosensitivity in A/W
I_det = P_det*photosens

print('Photocurrent = {:.3f} pA'.format(I_det*1e12))
print('Dark current = {:.3f} pA'.format(I_dark*1e12))

Photocurrent = 0.016 pA
Dark current = 5.000 pA
